In [9]:
import numpy as np
import pandas as pd
import yaml, os

def generate_inp_file(fname, title, cards, verbose=True, TITLE = False):
    with open("Others_inp/" + fname, 'w', newline='\n') as f:
        if TITLE:
            f.write(f'''*\n*---- {title}  ----\n*\n''')
        for line in cards.split('\n'):
            f.write(line.replace('\r\n', '\n').replace('\r','') + '\n')
    if verbose : 
        print(f' --- created file {fname} with {len(cards.split("\n"))} lines')
    return

## Note

I am currently operating in a hybrid FLAIR - FLUKA (.flair - .inp) framework  
  
In the future, change this to write the final .inp file WITHOUT FLAIR, and then  open it via flair.

# Load yaml file

In [2]:
with open('Others_inp/ESSnuSB.yaml', "r") as f:
    config = yaml.safe_load(f)

# Build Geometry

## Build Target & Horn

In [3]:
def create_target_and_horn(config=config):    
    ###########################################################
    ######################### BODIES ##########################
    ########################################################### 
    '''
    remarks :
     CaniSup = tgtbox // remove one or another to avoid repetition
    '''
    
    _bodies = f"""*\n* ----- Target and Horn bodies ----- \n*\n
RCC thbox    0.0 0.0 0.0  0.0 0.0 {config["Horn"]["L_box"]} {config["Horn"]["R_box"]}
RCC tgtbox   0.0 0.0 0.0  0.0 0.0 {config["Canister"]["L_sup"]} {config["Canister"]["R_sup"]}
RCC CaniSup  0.0 0.0 0.0  0.0 0.0 {config["Canister"]["L_sup"]} {config["Canister"]["R_sup"]}
RCC CaniInf  0.0 0.0 {config["Canister"]["w_skin"]}  0.0 0.0 {config["Canister"]["L_inf"]} {config["Canister"]["R_inf"]}
RCC CaniSkin 0.0 0.0 {config["Canister"]["w_skin"]}  0.0 0.0 {config["Canister"]["L_inf"]} {config["Canister"]["R_skin"]}
RCC target   0.0 0.0 {config["Canister"]["w_skin"]}  0.0 0.0 {config["Target"]["L"]} {config["Target"]["R"]}
"""
    for i in range (len(config["Horn"]["z"])):
        name = f"h0{i}" if i<10 else f"h{i}"
        _bodies += f'TRC {name}  0.0 0.0 {config["Horn"]["z"][i]} 0.0 0.0 {config["Horn"]["dz"][i]} {config["Horn"]["R0"][i]} {config["Horn"]["Rf"][i]}\n'

    
    ###########################################################
    ######################### REGIONS #########################
    ###########################################################
    _regions = f"""*\n* ----- Target and Horn regions ----- \n*\n
THBOX        5 +thbox
               -h00 -h01 -h02 -h03 -h04 -h05 -h06 -h07 -h08 -h09-tgtbox
RHALUP       5 +h00-h20 -h30 -h10-target |+h30 -h10-tgtbox
               |+h01-h21 -h31 -h11-target |+h31 -h11-tgtbox
               |+h02-h22 -h32 -h12-target |+h32 -h12-tgtbox
               |+h03-h23 -h33 -h13-target |+h33 -h13-tgtbox
               |+h04-h24 -h34 -h14-target |+h34 -h14-tgtbox
               |+h05-h25 -h35 -h15-target |+h35 -h15-tgtbox
               |+h06-h26 -h36 -h16-target |+h36 -h16-tgtbox
               |+h07-h27 -h37 -h17-target |+h37 -h17-tgtbox
               |+h08-h28 -h38 -h18-target |+h38 -h18-tgtbox
               |+h09-h29 -h39 -h19-target |+h39 -h19-tgtbox
RHBF         5 +h20-h30-tgtbox
               |+h21-h31-tgtbox
               |+h22-h32-tgtbox
               |+h23-h33-tgtbox
               |+h24-h34-tgtbox
               |+h25-h35-tgtbox
               |+h26-h36-tgtbox
               |+h27-h37-tgtbox
               |+h28-h38-tgtbox
               |+h29-h39-tgtbox
RHIN         5 +h10-tgtbox   |+h11-tgtbox   |+h12-tgtbox   |+h13-tgtbox  |+h14-tgtbox
               |+h15-tgtbox  |+h16 -tgtbox |+h17 -tgtbox |+h18 -tgtbox  |+h19-tgtbox
CANIBOX      5 +CaniSup -CaniInf | +CaniSkin -target
CANICOOL     5 +CaniInf -CaniSkin
TARGET       5 +target
"""
    
    ###########################################################
    ###################### MAT & ASSIGNMA #####################
    ###########################################################
    
    _mat = f"""*\n* ----- Target and Horn materials ----- \n*\n
MATERIAL         22.                  3.                              TITAN60
ASSIGNMA     TITAN60    TARGET
ASSIGNMA    TITANIUM   CANIBOX
ASSIGNMA      HELIUM  CANICOOL
ASSIGNMA      HELIUM     THBOX
ASSIGNMA    ALUMINUM    RHALUP
ASSIGNMA      HELIUM      RHBF                            1.
ASSIGNMA      HELIUM      RHIN
"""

    return _bodies, _regions, _mat

## Build Collimator (Shielding)

In [4]:
def create_clmt(config=config):    
    ###########################################################
    ######################### BODIES ##########################
    ########################################################### 
    x0, xf = config["Collimator"]["x"]
    y0, yf = config["Collimator"]["y"]
    z0, zf = config["Collimator"]["z"]
    Lclmt = config["Collimator"]["z"][1] - config["Collimator"]["z"][0]
    
    _bodies = f"""*\n* ----- Collimator bodies ----- \n*\n
RPP clmt     {x0} {xf} {y0} {yf} {z0} {zf}
RCC hclmt    0.0 0.0 {z0} 0.0 0.0 {Lclmt} {config["Collimator"]["R"]}
RCC clmtW    0.0 0.0 {z0} 0.0 0.0 {Lclmt} {config["Collimator"]["R_w"]}
"""
    ###########################################################
    ######################### REGIONS #########################
    ###########################################################
    _regions = f"""*\n* ----- Collimator regions ----- \n*\n
CLMT         5 +clmt -hclmt -clmtW
CLMTHOLE     5 +hclmt
CLMTW        5 +clmtW -hclmt
"""
    ###########################################################
    ###################### MAT & ASSIGNMA #####################
    ###########################################################
    
    _mat = f"""*\n* ----- Collimator materials ----- \n*\n
ASSIGNMA        IRON      CLMT
ASSIGNMA      HELIUM  CLMTHOLE
ASSIGNMA    TUNGSTEN     CLMTW
"""
    return _bodies, _regions, _mat

## Build Magnet

In [5]:
def create_dipole(config=config):    
    ###########################################################
    ######################### BODIES ##########################
    ########################################################### 
    x0, xf = config["Dipole"]["x"]
    y0, yf = config["Dipole"]["y"]
    z0, zf = config["Dipole"]["z"]

    xcoils = config["Dipole"]["xcoils"]
    ycoils = config["Dipole"]["ycoils"]

    w_ycoils = config["Dipole"]["w_ycoils"]
    y_airgap = [ycoils[0] + w_ycoils, ycoils[1] - w_ycoils]

    w_coils = config["Dipole"]["w_coils"]
    
    _bodies = f"""*\n* ----- Dipole bodies ----- \n*\n
RPP magnet    {x0} {xf} {y0} {yf} {z0} {zf}
RPP coils    {xcoils[0]} {xcoils[1]} {ycoils[0]} {ycoils[1]} {z0} {zf}
RPP airgap   {xcoils[0]} {xcoils[1]} {y_airgap[0]} {y_airgap[1]} {z0} {zf}
RPP Fecoils  {xcoils[0] + w_coils} {xcoils[1] - w_coils} {ycoils[0]} {ycoils[1]} {z0 + w_coils} {zf - w_coils}
"""
    ###########################################################
    ######################### REGIONS #########################
    ###########################################################
    _regions = f"""*\n* ----- Dipole regions ----- \n*\n
MAGNET       5 +magnet -airgap -(coils-Fecoils) 
COIL         5 +coils -Fecoils -airgap
BFIELD       5 +airgap
"""
    ###########################################################
    ###################### MAT & ASSIGNMA #####################
    ###########################################################
    
    _mat = f"""*\n* ----- Dipole materials ----- \n*\n
ASSIGNMA        IRON    MAGNET
ASSIGNMA      HELIUM    BFIELD                            1.
ASSIGNMA      COPPER      COIL
"""
    return _bodies, _regions, _mat

## Build Beam Dump and TTline

In [11]:
def compute_transformation(width, theta, Xf, Yf):
    '''
    compute the translation parameter (for the ttline) to be applied before rotation of theta degrees. 
    
    width : width of the t2r 
            should be given in [cm]
    theta : for ESSnuSB+, it should be negative (-29.5)
            should be given in [degree]
    Xf    : in the zx-plane, Xf refers to the ending point (after transformation) 
            on the z-axis of the left bottom corner of the t2r
            should be given in [cm]
    Yf    : in the zx-plane, Yf refers to the ending point (after transformation) 
            on the x-axis of the left bottom corner of the t2r
            should be given in [cm]        
    '''
    width = width*1e-2
    theta = np.deg2rad(theta)
    Zdif = width / np.sin(theta) 
    Xf = Xf*1e-2 + Zdif
    Yf = Yf*1e-2
    
    # initial coordinate of the left bottom corner point
    X0 = 0e-2
    Y0 = - width/2
    
    
    # Inversion of the rotation + translation
    '''
    R = rotation, R_1 = inverse rotation, T = translation
    X = initial coordinate, Y = final coordinate
    
    R = (+cos -sin)    R_1 = (+cos +sin)
        (+sin +cos)          (-sin +cos)

    Full transformation (ROTDEFI)
    Y = R(X+T)
    
    T = R_1(Y) - X
    --> T =
         (+xf cos + yf sin - x0)
         (-xf sin + yf cos - y0)
    xf, yf (final)
    x0, y0 (initial)
    '''
    R_1Yx = Xf * np.cos(theta) + Yf * np.sin(theta)
    R_1Yy = -Xf * np.sin(theta) + Yf * np.cos(theta)
    
    Trx = R_1Yx - X0
    Try = R_1Yy - Y0

    return Trx*100, Try*100

In [7]:
def create_BD_t2r(config=config):    
    ###########################################################
    ######################### BODIES ##########################
    ########################################################### 
    confbd = config["BeamDump"]
    x = [confbd["x"][0], confbd["x"][1]]
    y = [confbd["y"][0], confbd["y"][1]]
    z = [confbd["z0"], confbd["z0"] + confbd["width"]]

    conft2r = config["t2r"]
    width  = conft2r["width"]
    L = conft2r["length"]
    theta = conft2r["theta"]
    w_walls = conft2r["w_walls"] 
    w_vessel = conft2r["w_vessel"] 

    xwall = [-0.5*width - w_walls, 0.5*width + w_walls]
    ywall = [-0.5*width - w_walls, 0.5*width + w_walls]
    zwall = [0 - w_walls, L + w_walls]

    xvessel = [-0.5*width + w_vessel, 0.5*width - w_vessel]
    yvessel = [-0.5*width + w_vessel, 0.5*width - w_vessel]
    zvessel = [0 + w_vessel, L - w_vessel]

    Trx, Try = compute_transformation(width, theta, z[0], x[0])
    # in this configuration, the BeamDump is expected to touch the the wall of the Decay Tunnel
    # so x[0] should be -250.
    # Also, z[0] is the start of the BeamDump : I don't want the TTline to "touch it" 
    # (the apperture of the TTline is fully before the BeamDump

    #ROT-DEFI ttline 200 0 {np.abs(theta)} {Try:.2f} 0 {Trx:.2f}
    _ROTDEFI = f"""*\n* ----- TTline ROTDEFINE ----- \n*\n   
ROT-DEFI         200         0{np.abs(theta):>10}{Try:>10.4f}{0:>10}{Trx:>10.4f}ttline    
"""
    
    _bodies = f"""*\n* ----- BeamDump and TTline bodies ----- \n*\n 
RPP bdump    {x[0]} {x[1]} {y[0]} {y[1]} {z[0]} {z[1]}
$start_transform ttline 
RPP ttline   {-0.5*width} {0.5*width} {-0.5*width} {0.5*width} {0} {L}
RPP ttwall   {xwall[0]} {xwall[1]} {ywall[0]} {ywall[1]} {zwall[0]} {zwall[1]}
RPP ttvessel {xvessel[0]} {xvessel[1]} {yvessel[0]} {yvessel[1]} {zvessel[0]} {zvessel[1]}
$end_transform 
"""
    
    ###########################################################
    ######################### REGIONS #########################
    ###########################################################
    _regions = f"""*\n* ----- BeamDump and TTline regions ----- \n*\n 
BDUMP       5 +bdump 
TTLINE       5 +ttvessel -dt
"""
    ###########################################################
    ###################### MAT & ASSIGNMA #####################
    ###########################################################
    
    _mat = f"""*\n* ----- BeamDump and TTline materials ----- \n*\n 
ASSIGNMA      CARBON    BDUMP
ASSIGNMA      HELIUM   TTLINE
"""
    return _bodies, _regions, _mat, _ROTDEFI

## Saving cards - input files

In [13]:
# README

# For reasons that are not fully understood, when including an external .inp file
# using FLAIR, the content of the included file may be deleted.

# Solution:
# All input files should be included directly inside FLAIR using the
# #include / #endinclude mechanism.

# Please double-check that the correct file is actually included, as FLAIR may
# sometimes include a different file than the one selected.
# If this happens, repeat the operation until the correct file is included.

# Note:
# Inside the #include / #endinclude block, the content may appear empty in FLAIR
# (the file content is temporarily removed from the display).

# To restore the input files:
# - Save the FLAIR project (once the includes are set).
# - Regenerate the input files (for example using this notebook).
# - Reload the project when FLAIR asks for it.
#   If no prompt appears, reload the project manually by reopening it.

In [9]:
Generate_single_cards = True

    
th_bodies_cards, th_regions_cards, th_assignma_cards = create_target_and_horn(config)
CLMT_bodies_cards, CLMT_regions_cards, CLMT_assignma_cards = create_clmt(config)
Dipole_bodies_cards, Dipole_regions_cards, Dipole_assignma_cards = create_dipole(config)
BD_t2r_bodies_cards, BD_t2r_regions_cards, BD_t2r_assignma_cards, BD_t2r_ROTDEFI_cards= create_BD_t2r(config)

if Generate_single_cards:

    Bodies_cards = f""""""
    Regions_cards = f""""""
    Materials_cards = f""""""
    ROTDEFI_cards = f""""""

    # Creating variables with all the cards
    
    # Target & Horn
    Bodies_cards += th_bodies_cards
    Regions_cards += th_regions_cards
    Materials_cards += th_assignma_cards
    # CLMT
    Bodies_cards += CLMT_bodies_cards
    Regions_cards += CLMT_regions_cards
    Materials_cards += CLMT_assignma_cards
    # Dipole
    Bodies_cards += Dipole_bodies_cards
    Regions_cards += Dipole_regions_cards
    Materials_cards += Dipole_assignma_cards
    # BeamDump and TTline (t2r)
    Bodies_cards += BD_t2r_bodies_cards
    Regions_cards += BD_t2r_regions_cards
    Materials_cards += BD_t2r_assignma_cards
    ROTDEFI_cards += BD_t2r_ROTDEFI_cards

    # Saving cards into .inp files
    
    inp_name = "ESSnuSBp_bodies"
    generate_inp_file(f'{inp_name}.inp', None, Bodies_cards)
    os.chmod(f'Others_inp/{inp_name}.inp', 0o744) 
    inp_name = "ESSnuSBp_regions"
    generate_inp_file(f'{inp_name}.inp', None, Regions_cards)
    os.chmod(f'Others_inp/{inp_name}.inp', 0o744) 
    inp_name = "ESSnuSBp_materials"
    generate_inp_file(f'{inp_name}.inp', None, Materials_cards)
    os.chmod(f'Others_inp/{inp_name}.inp', 0o744) 
    inp_name = "ESSnuSBp_ROTDEFI"
    generate_inp_file(f'{inp_name}.inp', None, ROTDEFI_cards)
    os.chmod(f'Others_inp/{inp_name}.inp', 0o744) 
else:
    # Target and Horn
    generate_inp_file('thbox_bodies.inp', 'Target & Horn Assembly', th_bodies_cards)
    os.chmod('Others_inp/thbox_bodies.inp', 0o744)
    generate_inp_file('thbox_regions.inp', 'Target & Horn Assembly', th_regions_cards)
    os.chmod('Others_inp/thbox_regions.inp', 0o744)
    generate_inp_file('thbox_mat.inp', 'Target & Horn Materials', th_assignma_cards)
    os.chmod('Others_inp/thbox_mat.inp', 0o744) 
    # CLMT
    inp_name = "clmt"
    prefix = "Collimator"
    generate_inp_file(f'{inp_name}_bodies.inp', f'{prefix} Bodies', CLMT_bodies_cards)
    os.chmod(f'Others_inp/{inp_name}_bodies.inp', 0o744)
    generate_inp_file(f'{inp_name}_regions.inp', f'{prefix} Regions', CLMT_regions_cards)
    os.chmod(f'Others_inp/{inp_name}_regions.inp', 0o744)
    generate_inp_file(f'{inp_name}_mat.inp', f'{prefix} Materials', CLMT_assignma_cards)
    os.chmod(f'Others_inp/{inp_name}_mat.inp', 0o744) 
    # Dipole
    inp_name = "dipole"
    prefix = "Dipole"
    generate_inp_file(f'{inp_name}_bodies.inp', f'{prefix} Bodies', Dipole_bodies_cards)
    os.chmod(f'Others_inp/{inp_name}_bodies.inp', 0o744) 
    generate_inp_file(f'{inp_name}_regions.inp', f'{prefix} Regions', Dipole_regions_cards)
    os.chmod(f'Others_inp/{inp_name}_regions.inp', 0o744)
    generate_inp_file(f'{inp_name}_mat.inp', f'{prefix} Materials', Dipole_assignma_cards)
    os.chmod(f'Others_inp/{inp_name}_mat.inp', 0o744) 
    # Bd & t2r
    inp_name = "BD_t2r"
    prefix = "BD_t2r"
    generate_inp_file(f'{inp_name}_bodies.inp', f'{prefix} Bodies', BD_t2r_bodies_cards)
    os.chmod(f'Others_inp/{inp_name}_bodies.inp', 0o744)
    generate_inp_file(f'{inp_name}_regions.inp', f'{prefix} Regions', BD_t2r_regions_cards)
    os.chmod(f'Others_inp/{inp_name}_regions.inp', 0o744)
    generate_inp_file(f'{inp_name}_mat.inp', f'{prefix} Materials', BD_t2r_assignma_cards)
    os.chmod(f'Others_inp/{inp_name}_mat.inp', 0o744) 
    generate_inp_file(f'{inp_name}_ROTDEFI.inp', f'{prefix} ROTDEFI', BD_t2r_ROTDEFI_cards)
    os.chmod(f'Others_inp/{inp_name}_ROTDEFI.inp', 0o744) 

 --- created file ESSnuSBp_bodies.inp with 76 lines
 --- created file ESSnuSBp_regions.inp with 52 lines
 --- created file ESSnuSBp_materials.inp with 33 lines
 --- created file ESSnuSBp_ROTDEFI.inp with 6 lines
